In [ ]:
# Import library yang diperlukan
from bs4 import BeautifulSoup
import requests
import csv
import time

# Mendefinisikan fungsi detik dengan parameter keyword, n_berita, max_pages, dan delay_time
def detik(keyword=None, n_berita=None, max_pages=None, delay_time=None):
    # Memeriksa apakah semua parameter telah disediakan
    if keyword and n_berita and max_pages and delay_time:
        # Membuat URL pencarian detik berdasarkan keyword
        alamatURL = 'https://www.detik.com/search/searchall?query=anies+muhaimin&siteid=2&sortby=time&sorttime=0&result_type=relevansi&page'
        data_detik = []  # Inisialisasi list untuk menyimpan data berita
        pakem = True  # Variabel kontrol untuk menentukan apakah proses scraping harus berlanjut
        current_page = 1  # Halaman saat ini

        # Melakukan scraping selama kondisi pakem dan halaman kurang dari atau sama dengan max_pages
        while pakem and current_page <= max_pages:
            try:
                # Mengambil halaman HTML dari URL detik
                req = requests.get(alamatURL)
                req.raise_for_status()

                # Mengolah halaman HTML menggunakan BeautifulSoup
                page_source = req.content
                soup = BeautifulSoup(page_source, 'lxml')

                # Mengabaikan artikel dengan class foto_tag atau video_tag
                classToIgnore = ["foto_tag", "video_tag"]
                articles = soup.find_all('article', class_=lambda x: x not in classToIgnore)

                # Iterasi melalui artikel-artikel yang ditemukan
                for article in articles:
                    judul = article.find('h2', {'class': 'title'})
                    url = article.find('a', href=True)['href']

                    # Mengambil detail dari setiap artikel
                    req_detail = requests.get(url)
                    req_detail.raise_for_status()

                    # Mengolah halaman detail artikel menggunakan BeautifulSoup
                    soup_detail_berita = BeautifulSoup(req_detail.content, 'lxml')
                    berita_element = soup_detail_berita.find('div', {'class': 'detail__body-text itp_bodycontent'})

                    # Memeriksa keberadaan judul dan isi berita
                    if judul and berita_element:
                        date = article.find('span', {'class': 'date'}).get_text()

                        # Menyimpan data artikel dalam bentuk dictionary
                        data = {
                            'judul': judul.get_text(),
                            'berita': ''.join(berita_element.get_text().splitlines()),
                            'url': url,
                            'tanggal': date.split(',')[1].strip()
                        }

                        # Menambahkan data ke dalam list data_detik
                        data_detik.append(data)
                        print('Judul berita : %s' % judul.get_text())
                        print('Total berita yang didapat: %s' % len(data_detik))
                        print('================================================')

                        # Memeriksa apakah sudah mencapai jumlah berita yang diinginkan
                        if len(data_detik) >= int(n_berita):
                            pakem = False
                            break

                # Mencari link untuk halaman berikutnya
                next_page_link = soup.find('a', {'onclick': '_pt(this, "pagination", "button pagination", "pagination Next")'})
                if next_page_link:
                    next_page_url = next_page_link['href']
                    alamatURL = next_page_url
                    time.sleep(delay_time)  # Tambahkan penundaan antar permintaan
                    current_page += 1
                else:
                    pakem = False

            except requests.RequestException as e:
                print("Kesalahan permintaan:", str(e))
                time.sleep(5)  # Tambahkan penundaan sebelum mencoba lagi

            except Exception as e:
                print("Kesalahan yang tidak terduga terjadi:", str(e))
                pakem = False

        # Ekspor data ke dalam file CSV
        keys = data_detik[0].keys()
        with open('detik_%s_%s.csv' % (keyword, n_berita), 'w', newline='', encoding='utf-8') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data_detik)
            print('Data tersimpan di detik_%s_%s.csv' % (keyword, n_berita))

    else:
        # Pesan yang akan ditampilkan jika parameter tidak disediakan
        print('''
        Masukkan keyword, jumlah berita, jumlah halaman maksimal, dan waktu delay antar permintaan
        contoh:
        python detik_scraper.py amin 1000 20 1
        ''')

# Eksekusi scraper dengan input contoh
try:
    KEYWORD = 'anies muhaimin'
    N_BERITA = '2000'
    MAX_PAGES = 300
    DELAY_TIME = 0.1  # Waktu delay dalam detik
    detik(keyword=KEYWORD, n_berita=N_BERITA, max_pages=MAX_PAGES, delay_time=DELAY_TIME)
except IndexError:
    # Pesan yang akan ditampilkan jika input tidak sesuai
    print('''
    Masukkan keyword, jumlah berita, jumlah halaman maksimal, dan waktu delay antar permintaan
    contoh:
    python detik_scraper.py amin 1000 20 1
    ''')


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
from IPython.display import clear_output

upperframe=[]
for page in range(1, 5):
    print('processing page :', page)

    # Link website yang akan discrap
    #url = 'https://www.kompas.com/tag/covid-19?sort=desc&page='+str(page)
    #url = 'https://www.kompas.com/tag/karantina?sort=asc&page='+str(page)
    #url = 'https://www.kompas.com/tag/isolasi?sort=asc&page='+str(page)
    #url = 'https://www.kompas.com/tag/kasus+covid-19+?sort=asc&page='+str(page)
    url = 'https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page='+str(page)
    print(url)

    try:
        page=requests.get(url)

    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print ('ERROR FOR LINK:',url)
        print (error_type, 'Line:', error_info.tb_lineno)
        continue

    #Kasih Jeda request ke Web
    time.sleep(0.1)

    soup = BeautifulSoup(page.text, "html.parser")
    frame=[]
    Article = soup.find_all('div',attrs={'class':'article__list clearfix'})

    for j in Article:
        #Ambil Judul
        Judul = j.find('a',attrs={'class':'article__link'}).text.strip()
        #Ambil Link
        Links = j.find('a')['href'].strip()+"?page=all" #Biar langsung page all
        #Ambil Waktu
        Date = j.find('div', attrs={'class':'article__date'}).text.strip()

        #Arahkan ke link yang tadi diambil
        try:
            inside = requests.get(Links)
            #Ambil isi konten berdasarkan class
            soup_inside = BeautifulSoup(inside.text, "html.parser")
            Contents = soup_inside.find('div',attrs={'class':'read__content'}).text.strip()

        except Exception as e:
            error_type, error_obj, error_info = sys.exc_info()
            print ('ERROR FOR LINK:',url)
            print (error_type, 'Line:', error_info.tb_lineno)
            continue

        # Masukkan data yang telah didapat kedalam satu baris tabel
        frame.append((Judul,Links,Date,Contents))
    # Kumpulkan baris baris yang telah didapat menjadi sebuat tabel
    upperframe.extend(frame)

#Buat nama kolom tabel
data=pd.DataFrame(upperframe, columns=['Judul','Isi Berita','Link','Waktu'])
#Tes munculkan 5 baris pertama tabel
data.head()

processing page : 1
https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page=1
processing page : 2
https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page=2
processing page : 3
https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page=3
processing page : 4
https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page=4


,Judul,Isi Berita,Link,Waktu
0,"Pandemi Covid-19, Atta Halilintar Galang Dana ...",http://www.kompas.com/hype/read/2020/03/18/155...,"18/03/2020, 15:55 WIB","JAKARTA, KOMPAS.com - Peraturan pemerintah pen..."
1,China Siap Bantu Indonesia Hadapi Pandemi Covi...,http://nasional.kompas.com/read/2020/03/19/074...,"19/03/2020, 07:47 WIB","JAKARTA, KOMPAS.com - YouTuber Atta Halilintar..."
2,Ombudsman Minta Kualitas Layanan Publik Tak Tu...,http://nasional.kompas.com/read/2020/03/19/155...,"19/03/2020, 15:53 WIB","JAKARTA, KOMPAS.com - Duta Besar China untuk I..."
3,"Pandemi Covid-19, Pemkot Depok Minta Umat Tund...",http://megapolitan.kompas.com/read/2020/03/19/...,"19/03/2020, 19:48 WIB","JAKARTA, KOMPAS.com - Ombudsman Republik Indon..."
4,"Hadapi Pandemi Covid-19, Kementan Gandeng Pema...",http://money.kompas.com/read/2020/03/20/155000...,"20/03/2020, 15:50 WIB","DEPOK, KOMPAS.com - Pemerintah Kota Depok meng..."


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
from IPython.display import clear_output

upperframe=[]
for page in range(1, 5):
    print('processing page :', page)

    # Link website yang akan discrap
    #url = 'https://www.kompas.com/tag/covid-19?sort=desc&page='+str(page)
    #url = 'https://www.kompas.com/tag/karantina?sort=asc&page='+str(page)
    #url = 'https://www.kompas.com/tag/isolasi?sort=asc&page='+str(page)
    #url = 'https://www.kompas.com/tag/kasus+covid-19+?sort=asc&page='+str(page)
    url = 'https://www.kompas.com/tag/pandemi+covid-19?sort=asc&page='+str(page)
    print(url)

    try:
        page=requests.get(url)

    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print ('ERROR FOR LINK:',url)
        print (error_type, 'Line:', error_info.tb_lineno)
        continue

    #Kasih Jeda request ke Web
    time.sleep(0.1)

    soup = BeautifulSoup(page.text, "html.parser")
    frame=[]
    Article = soup.find_all('div',attrs={'class':'article__list clearfix'})

    for j in Article:
        #Ambil Judul
        Judul = j.find('a',attrs={'class':'article__link'}).text.strip()
        #Ambil Link
        Links = j.find('a')['href'].strip()+"?page=all" #Biar langsung page all
        #Ambil Waktu
        Date = j.find('div', attrs={'class':'article__date'}).text.strip()

        #Arahkan ke link yang tadi diambil
        try:
            inside = requests.get(Links)
            #Ambil isi konten berdasarkan class
            soup_inside = BeautifulSoup(inside.text, "html.parser")
            Contents = soup_inside.find('div',attrs={'class':'read__content'}).text.strip()

        except Exception as e:
            error_type, error_obj, error_info = sys.exc_info()
            print ('ERROR FOR LINK:',url)
            print (error_type, 'Line:', error_info.tb_lineno)
            continue

        # Masukkan data yang telah didapat kedalam satu baris tabel
        frame.append((Judul,Links,Date,Contents))
    # Kumpulkan baris baris yang telah didapat menjadi sebuat tabel
    upperframe.extend(frame)

#Buat nama kolom tabel
data=pd.DataFrame(upperframe, columns=['Judul','Isi Berita','Link','Waktu'])
#Tes munculkan 5 baris pertama tabel
data.head()

In [33]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
from IPython.display import clear_output

upperframe = []
for page_num in range(1, 4):  # Sesuaikan range sesuai kebutuhan
    print('Processing page:', page_num)

    # Link website yang akan discrap
    url = 'https://www.kompas.com/tag/pilpres+indonesia+2024?sort=asc&page=' + str(page_num)
    print(url)

    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print('ERROR FOR LINK:', url)
        print(error_type, 'Line:', error_info.tb_lineno)
        continue

    # Kasih Jeda request ke Web
    time.sleep(0.1)

    soup = BeautifulSoup(page.text, "html.parser")
    frame = []
    articles = soup.find_all('div', attrs={'class': 'article__list clearfix'})

    for article in articles:
        # Ambil Judul
        title_element = article.find('a', attrs={'class': 'article__link'})
        if title_element:
            Title = title_element.text.strip()
        else:
            Title = 'Judul tidak ditemukan'

        # Ambil Link
        link_element = article.find('a', attrs={'class': 'article__link'})
        if link_element:
            Links = link_element['href'].strip() + "?page=all"  # Biar langsung page all
        else:
            Links = 'Link tidak ditemukan'

        # Ambil Waktu
        date_element = article.find('div', attrs={'class': 'article__date'})
        if date_element:
            Date = date_element.text.strip()
        else:
            Date = 'Tanggal tidak ditemukan'

        # Arahkan ke link yang tadi diambil
        try:
            if Links != 'Link tidak ditemukan':
                inside = requests.get(Links)
                # Kasih Jeda request ke Web
                time.sleep(0.1)
                # Ambil isi konten berdasarkan class
                soup_inside = BeautifulSoup(inside.text, "html.parser")
                content_element = soup_inside.find('div', attrs={'class': 'read__content'})
                if content_element:
                    Contents = content_element.text.strip()
                else:
                    Contents = 'Konten tidak ditemukan'

                # Ambil tanggal publikasi dari halaman dalam (jika ada)
                publish_date_element = soup_inside.find('div', attrs={'class': 'read__time'})
                if publish_date_element:
                    PublishDate = publish_date_element.text.strip()
                else:
                    PublishDate = Date  # Menggunakan tanggal dari halaman utama jika tidak ditemukan
            else:
                Contents = 'Konten tidak ditemukan'
                PublishDate = Date
        except Exception as e:
            error_type, error_obj, error_info = sys.exc_info()
            print('ERROR FOR LINK:', Links)
            print(error_type, 'Line:', error_info.tb_lineno)
            continue

        # Masukkan data yang telah didapat kedalam satu baris tabel
        frame.append((Title, Contents, Links, PublishDate))

    # Kumpulkan baris-baris yang telah didapat menjadi sebuah tabel
    upperframe.extend(frame)

# Buat nama kolom tabel
data = pd.DataFrame(upperframe, columns=['Judul', 'Isi Berita', 'Link', 'Tanggal Publikasi'])
# Simpan dalam file csv
data.to_csv('scraped_data_pilpres_2024.csv', index=False)
data.head()


Processing page: 1
https://search.kompas.com/search/?q=pilpres+indonesia+2024&submit=Submit#gsc.tab=0&gsc.q=pilpres%20indonesia%202024&gsc.page=1
Processing page: 2
https://search.kompas.com/search/?q=pilpres+indonesia+2024&submit=Submit#gsc.tab=0&gsc.q=pilpres%20indonesia%202024&gsc.page=2
Processing page: 3
https://search.kompas.com/search/?q=pilpres+indonesia+2024&submit=Submit#gsc.tab=0&gsc.q=pilpres%20indonesia%202024&gsc.page=3
Empty DataFrame
Columns: [Judul, Isi Berita, Link, Tanggal Publikasi]
Index: []
